In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Data
df = pd.read_excel("/Users/farrelmanazilin/Downloads/data/gemas.xlsx")
df = df['Consumption Goods']
n = len(df)

# Split data into training and testing sets
train_size = int(0.5 * n)
train_data = df[:train_size]
test_data = df[train_size:]

# Holt-Winters Method without Seasonality
def holt_winters(data, alpha, beta, iterations):
    level = data[0]
    trend = data[1] - data[0]
    forecast = []

    for i in range(len(data) + iterations):
        if i >= len(data):
            forecast.append(level + trend * (i - len(data) + 1))
        else:
            forecast.append(level + trend)
            level = alpha * data[i] + (1 - alpha) * (level + trend)
            trend = beta * (level - data[i]) + (1 - beta) * trend

    return forecast

# Set Holt-Winters parameters
alpha = 0.6
beta = 0.6
iterations = len(test_data)

# Apply Holt-Winters method to forecast
forecast = holt_winters(train_data, alpha, beta, iterations)

# Extend train_data with NaN values for the forecast period
extended_train_data = pd.concat([train_data, pd.Series(np.nan, index=test_data.index)])

# Slice the forecast array for the forecast period
forecast_sliced = forecast[:len(test_data)]

# Replace NaN values in extended_train_data with forecast values
extended_train_data.loc[test_data.index] = forecast_sliced

# Calculate MAE and RMSE
mae = mean_absolute_error(test_data, forecast_sliced)
rmse = np.sqrt(mean_squared_error(test_data, forecast_sliced))

# Calculate Error
error = test_data - forecast_sliced

# Create a DataFrame for Demand, Forecast, and Error
results = pd.DataFrame({'Demand': test_data.values, 'Forecast': forecast_sliced, 'Error': error})

# Display the table
print(results)

# Print MAE and RMSE
print('MAE:', mae)
print('RMSE:', rmse)

# Plot actual data and forecast
plt.plot(df, label='Actual Data')
plt.plot(extended_train_data, label='Forecast')
plt.legend()
plt.show()